# 🦤 Dodo Source

[doit](https://github.com/pydoit/doit) is a simple, yet powerful task execution tool,
written in Python. The `DoitSource` demonstrates.

In [ ]:
if __name__ == "__main__" and "pyodide" in __import__("sys").modules:
    %pip install -q -r requirements.txt

In [ ]:
import ipywidgets as W
import traitlets as T
from ipydatagrid import DataGrid
from ipyforcegraph import behaviors as B
from ipyforcegraph import graphs as G
from ipyforcegraph.sources.dodo import DodoSource
from ipylab import JupyterFrontEnd, Panel

## Create a Panel

In [ ]:
app = JupyterFrontEnd()
panel = Panel([])
app.shell.add(panel, "main", {"mode": "split-right"})

## Create some Behaviors

In [ ]:
n_sel, l_sel = B.NodeSelection(), B.LinkSelection()
n_tooltip = B.NodeTooltip(
    B.Nunjucks("<b>[{{ node.type }}]</b> {{ node.name }}<br/>{{ node.doc }}")
)
l_tooltip = B.LinkTooltip(
    B.Nunjucks(
        "{{ link.source.name }}<br/><b>[{{ link.type }}]</b> <br/>{{ link.target.name }}"
    )
)

## Create the Source

In [ ]:
source = DodoSource(project_root="..")
T.dlink((source, "project_root"), (panel.title, "label"), lambda x: x.name);

## Create the Graph

In [ ]:
fg = G.ForceGraph(
    source=source,
    layout=dict(height="100%"),
    behaviors=[n_sel, l_sel, n_tooltip, l_tooltip],
)
panel.children = [fg]

## Create some Grids

In [ ]:
grid_opts = dict(
    selection_mode="row",
    layout=dict(height="100%", max_height="100%", min_height="400px"),
)
n_grid = DataGrid(fg.source.nodes, **grid_opts)
l_grid = DataGrid(fg.source.links, **grid_opts)
panel.children = [fg, W.HBox([n_grid, l_grid])]

## Link the Grids

In [ ]:
def link_grid_and_graph(grid, sel):
    def on_grid_select(*x):
        if not grid.selections:
            n_sel.selected = []
            return
        visible = grid.get_visible_data()
        s0 = grid.selections[0]
        sel.selected = [int(visible.iloc[i].name) for i in range(s0["r1"], s0["r2"])]

    grid.observe(on_grid_select, ["selections"])

In [ ]:
link_grid_and_graph(n_grid, n_sel)
link_grid_and_graph(l_grid, l_sel)